CC5213 - Recuperación de Información Multimedia
==

Ejemplos OpenCV
==

Una forma simple para instalar Python es usar Anaconda https://www.anaconda.com/download/

* En Linux:
  * Descargar `Anaconda3-2019.07-Linux-x86_64.sh` e instalar en la ruta `$HOME/anaconda3`
  * Modificar `.bashrc` y agregar al PATH  `$HOME/anaconda3/bin`
* En Windows:
  * Descargar `Anaconda3-2019.07-Windows-x86_64.exe` e instalar en la ruta `C:\[YYY]\Anaconda3`
  * Agregar al PATH `C:\[YYY]\Anaconda3` y `C:\[YYY]\Anaconda3\Scripts`

Existen variados entornos para programar en Python. Anaconda incluye **Spyder** y **Jupyter**.

Para iniciar Jupyter desde un terminal:

```
jupyter notebook --no-browser --port=8569
```

Instalar librerías requeridas:

```
conda install opencv
conda install -c conda-forge opencv
```

**NOTA**: Si ocurre un conflicto y no se puede instalar opencv se debe crear un environment para python 3.6.

In [1]:
import sys
import os
import numpy
import cv2

print ("Usando OpenCV {} Python {}.{}.{}".format(cv2.__version__, sys.version_info.major, sys.version_info.minor, sys.version_info.micro))

## Declaracion de algunas funciones utiles
def mostrar_imagen(window_name, imagen, valorAbsoluto = False, escalarMin0Max255 = False):
    if valorAbsoluto:
        imagen_abs = numpy.abs(imagen)
    else:
        imagen_abs = imagen
    if escalarMin0Max255:
        imagen_norm = cv2.normalize(imagen_abs, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    else:
        imagen_norm = imagen_abs
    #reducir tamaño
    MAX_WIDTH=500
    MAX_HEIGHT=400
    if imagen_norm.shape[0] > MAX_HEIGHT or imagen_norm.shape[1] > MAX_WIDTH:
        f0 = MAX_HEIGHT / imagen_norm.shape[0]
        f1 = MAX_WIDTH / imagen_norm.shape[1]
        imagen_norm = cv2.resize(imagen_norm, (0,0), fx=min(f0,f1), fy=min(f0,f1))
    #mostrar en pantalla
    cv2.imshow(window_name, imagen_norm)

def print_text(imagen, texto):
    fontFace = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 1
    thickness = 2
    textSize = cv2.getTextSize(texto, fontFace, fontScale, thickness)
    position1 = (20,20)
    position2 = (30,30)
    cv2.rectangle(imagen, position1, position2, (0,255,0), -1)
    cv2.putText(imagen, texto, (20,20), fontFace, fontScale, (255,0,0), thickness, cv2.LINE_AA)

def abrir_video(filename):
    print ("abriendo '{}'...".format(filename))
    if filename is None:
        filename = 0
    elif filename.isdigit():
        filename = int(filename)
    capture = None
    if isinstance(filename, int):
        capture = cv2.VideoCapture(filename)
    if (os.path.isfile(filename)):
        capture = cv2.VideoCapture(filename)
    if capture is None or not capture.isOpened():
        raise Exception("no puedo abrir video '{}'".format(filename))
    return capture;

import tkinter
import tkinter.filedialog

def escoger_imagen():
    root = tkinter.Tk()
    root.focus_force()
    filename = tkinter.filedialog.askopenfilename(parent=root, title="Escoger Imagen", filetypes=[("imagenes","*.jpg *.png *.jpeg")])
    root.destroy()
    return filename

def escoger_video():
    root = tkinter.Tk()
    root.focus_force()
    filename = tkinter.filedialog.askopenfilename(parent=root, title="Escoger Video", filetypes=[("videos","*.mp4 *.mpg *.avi")])
    root.destroy()
    if filename == "":
        return "0"
    return filename


Usando OpenCV 4.2.0 Python 3.7.6


Ejemplo 1 - OTSU
==


In [2]:
def ejemplo(filename):
    print ("abriendo {}".format(filename))
    imagen_color = cv2.imread(filename, cv2.IMREAD_COLOR)
    if imagen_color is None:
        print ("error abriendo {}".format(filename))
        return
    imagen_gris = cv2.cvtColor(imagen_color, cv2.COLOR_BGR2GRAY)
    threshold, imagen_bin = cv2.threshold(imagen_gris, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    window_name = os.path.basename(filename)
    cv2.imshow(window_name, imagen_color)
    cv2.imshow(window_name + " GRIS", imagen_gris)
    cv2.imshow(window_name + " BINARIA", imagen_bin)
    print ("{}: size={} threshold_otsu={}".format(window_name, imagen_color.shape, threshold))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

filename = escoger_imagen()

if filename != "":
    ejemplo(filename)
    
print("FIN")

abriendo C:/cursos/imagenes/entel2.png
entel2.png: size=(496, 717, 3) threshold_otsu=179.0
FIN


Ejemplo 2 - Sobel
==

In [3]:
def ejemplo(filename):
    global sobel_threshold, delta
    capture = abrir_video(filename)
    while capture.grab():
        retval, frame = capture.retrieve()
        if not retval:
            continue
        #convertir a gris
        frame_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        mostrar_imagen("VIDEO", frame_gris)
        #calcular filtro de sobel
        sobelX = cv2.Sobel(frame_gris, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=3)
        sobelY = cv2.Sobel(frame_gris, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=3)
        mostrar_imagen("X", sobelX, valorAbsoluto=True, escalarMin0Max255=True)
        mostrar_imagen("Y", sobelY, valorAbsoluto=True, escalarMin0Max255=True)
        #magnitud del gradiente
        magnitud = numpy.sqrt( numpy.square(sobelX) + numpy.square(sobelY) )
        mostrar_imagen("MAGNITUD GRADIENTE", magnitud, escalarMin0Max255=True)
        #aproximacion de la magnitud del gradiente
        magnitud_aprox = numpy.abs(sobelX) + numpy.abs(sobelY)
        mostrar_imagen("APROX GRADIENTE", magnitud_aprox, escalarMin0Max255=True)
        #umbral sobre la magnitud del gradiente
        retval, bordes = cv2.threshold(magnitud, thresh=sobel_threshold, maxval=255, type=cv2.THRESH_BINARY)
        print_text(bordes, "th={}".format(sobel_threshold))
        mostrar_imagen("BORDES", bordes)
        #esperar por una tecla
        key = cv2.waitKey(10)
        if key == ord(' '):
            key = cv2.waitKey(0)
        if key == ord('q') or key == 27:
            break
        elif key == ord('a'):
            sobel_threshold += delta
        elif key == ord('z'):
            if sobel_threshold - delta > 0:
                sobel_threshold -= delta
    capture.release()
    cv2.destroyAllWindows()

sobel_threshold = 51
delta = 5
filename = escoger_video()

if filename != "":
    ejemplo(filename)

print("FIN")

abriendo '0'...
FIN


Ejemplo 3 - CANNY
==

In [4]:
def ejemplo(filename):
    global canny_threshold_1, canny_threshold_2, delta
    capture = abrir_video(filename)
    while capture.grab():
        retval, frame = capture.retrieve()
        if not retval:
            continue
        #convertir a gris
        frame_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        mostrar_imagen("VIDEO", frame_gris)
        #calcular canny
        frame_canny = cv2.Canny(frame_gris, threshold1=canny_threshold_1, threshold2=canny_threshold_2)
        print_text(frame_canny, "th={}-{}".format(canny_threshold_1, canny_threshold_2))
        mostrar_imagen("CANNY", frame_canny)
        #esperar por una tecla
        key = cv2.waitKey(10)
        if key == ord(' '):
            key = cv2.waitKey(0)
        if key == -1:
            continue
        elif key == ord('q') or key == 27:
            break
        elif key == ord('a'):
            canny_threshold_1 += delta
        elif key == ord('z'):
            if canny_threshold_1 - delta > 0:
                canny_threshold_1 -= delta
        elif key == ord('s'):
            canny_threshold_2 += delta
        elif key == ord('x'):
            if canny_threshold_2 - delta > 0:
                canny_threshold_2 -= delta
        else:
            print("unknown key '{}' ({})".format(chr(key), key))
    capture.release()
    cv2.destroyAllWindows()

canny_threshold_1 = 110
canny_threshold_2 = 210
delta = 50
filename = escoger_video()

if filename != "":
    ejemplo(filename)

print("FIN")



abriendo '0'...
FIN


Ejemplo 4 - Difference of Gaussians
==

In [5]:
def ejemplo(filename):
    global sigma1, sigma2, threshold
    capture = abrir_video(filename)
    while capture.grab():
        retval, frame = capture.retrieve()
        if not retval:
            continue
        #convertir a gris
        frame_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        mostrar_imagen("VIDEO", frame_gris)
        #calcular DoG
        blur1 = cv2.GaussianBlur(frame_gris, (sigma1, sigma1), 0)
        blur2 = cv2.GaussianBlur(frame_gris, (sigma2, sigma2), 0)
        frame_diff = cv2.subtract(blur1, blur2)
        mostrar_imagen("Diff", frame_diff, escalarMin0Max255=True)
        th, frame_bin = cv2.threshold(frame_diff, threshold, 255, cv2.THRESH_BINARY)
        print_text(frame_bin, "th={}".format(threshold))
        mostrar_imagen("BIN", frame_bin, escalarMin0Max255=True)
        #esperar por una tecla
        key = cv2.waitKey(10)
        if key == ord(' '):
            key = cv2.waitKey(0)
        if key == -1:
            continue
        elif key == ord('q') or key == 27:
            break
        elif key == ord('a'):
            threshold += delta
        elif key == ord('z'):
            if threshold - delta > 0:
                threshold -= delta
    capture.release()
    cv2.destroyAllWindows()

sigma1 = 3
sigma2 = 13
threshold = 3
delta = 0.05

filename = escoger_video()

if filename != "":
    ejemplo(filename)

print("FIN")


abriendo '0'...
FIN
